In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from utils import *


In [8]:
directory = '../images/final/'
width = 96
height = 96
channels = 4
image_size = (width, height)
input_shape = (width, height, channels)
batch_size = 32
latent_dim = 100
num_classes = 18


In [9]:
train_ds, class_names = load_data(directory, batch_size, image_size, GAN=True)


Found 13849 files belonging to 905 classes.


In [10]:
types = pd.read_csv('../PokeDataset.csv', usecols=['Name', 'Type1', 'Type2'])
# reorder by alphabetical order of column 'Name'
types = types.sort_values(by=['Name'])
types = types.reset_index(drop=True)


In [11]:
# give percentage of pokemon with double type
print(
    f"Percentage of pokemon with double type: {types['Type2'].count() / types['Type1'].count() * 100:.2f}%")


Percentage of pokemon with double type: 50.69%


In [12]:
types.head(5)


,Name,Type1,Type2
0,Abomasnow,Grass,Ice
1,Abra,Psychic,NaN
2,Absol,Dark,NaN
3,Accelgor,Bug,NaN
4,Aegislash,Steel,Ghost


In [13]:
# make a dictionary of types and their index in the list of types
types_dict = {}
for i, t in enumerate(types['Type1'].unique()):
    types_dict[t] = i

# create vector of labels with 0 for each type except the one in the row (type1 and type2 are strings)


def create_label(row):
    label = np.zeros(18)
    label[types_dict[row['Type1']]] = 1
    if row['Type2'] in types_dict:
        label[types_dict[row['Type2']]] = 1
    return label


types['Label'] = types.apply(create_label, axis=1)


In [14]:
def types_to_vector(type1, type2=None):
    label = np.zeros(18)
    label[types_dict[type1]] = 1
    if type2 in types_dict:
        label[types_dict[type2]] = 1
    return label.reshape(1, 18)


In [15]:
for x, y in train_ds.take(1):
    print(int(tf.argmax(tf.reshape(y[0], [-1, 1]), axis=0)))
    plot_image(x[0], class_names[int(
        tf.argmax(tf.reshape(y[0], [-1, 1]), axis=0))], size=width)


521


In [16]:
# map labels in train_ds to the column Label in types
def map_labels(x, y):
    y_onehot = tf.argmax(y, axis=1).numpy()
    y_labels = np.zeros((len(y_onehot), 18))
    for i in range(len(y_onehot)):
        y_labels[i] = types.loc[y_onehot[i], 'Label']
    y_label = np.insert(y_labels, 0, 1, axis=1)
    y_label_tensor = tf.convert_to_tensor(y_label, dtype=tf.float32)
    return x, y_label_tensor


In [17]:
train_ds = prepare(train_ds, shuffle=True, augment=True, GAN=True)
train_ds = train_ds.map(lambda x, y: tf.py_function(
    map_labels, [x, y], [tf.float32, tf.float32]))

channels = 3
input_shape = (width, height, channels)


In [12]:
plot_n_images(train_ds, 5, class_names, GAN=True, size=width)


tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(19,), dtype=float32)
(96, 96, 3)


tf.Tensor([1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(19,), dtype=float32)
(96, 96, 3)


tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.], shape=(19,), dtype=float32)
(96, 96, 3)


tf.Tensor([1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(19,), dtype=float32)
(96, 96, 3)


tf.Tensor([1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(19,), dtype=float32)
(96, 96, 3)


In [18]:
from keras.layers import Dense, Reshape, Conv2DTranspose, Activation, BatchNormalization, Input, LeakyReLU, Conv2D, BatchNormalization, Flatten, Dropout
from keras.models import Model, Sequential


def build_encoder(input_shape, latent_dim):
    input_layer = Input(shape=input_shape)

    x = Conv2D(filters=32, kernel_size=3, strides=2,
               padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(units=latent_dim)(x)

    output_layer = x

    model = Model(inputs=input_layer, outputs=output_layer)

    return model


def build_generator(latent_dim):
    inputs = Input(shape=(latent_dim,))
    x = Dense(12*12*256)(inputs)
    x = Reshape((12, 12, 256))(x)
    x = Conv2DTranspose(128, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(64, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(3, kernel_size=5, strides=2, padding='same')(x)
    outputs = Activation('tanh')(x)
    generator = Model(inputs=inputs, outputs=outputs, name='generator')
    return generator


def build_discriminator_image(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2),
              padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model


def build_discriminator_latent(input_shape):
    model = Sequential()
    model.add(Dense(256, input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [19]:
encoder = build_encoder(input_shape, latent_dim)
generator = build_generator(latent_dim)
discriminator_image = build_discriminator_image(input_shape)
discriminator_latent = build_discriminator_latent((latent_dim,))


In [24]:
class AEGAN():
    def __init__(self, encoder, generator, discriminator_image, discriminator_latent, latent_dim, input_shape):
        self.encoder = encoder
        self.generator = generator
        self.discriminator_image = discriminator_image
        self.discriminator_latent = discriminator_latent
        self.latent_dim = latent_dim
        self.input_shape = input_shape

        self.discriminator_image.compile(
            loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.discriminator_latent.compile(
            loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.encoder.trainable = False
        self.generator.trainable = False

        z = Input(shape=(self.latent_dim,))
        fake_image = self.generator(z)
        fake_latent = self.encoder(fake_image)
        valid_image = self.discriminator_image(fake_image)
        valid_latent = self.discriminator_latent(fake_latent)

        self.combined = Model(z, [valid_image, valid_latent])
        self.combined.compile(
            loss=['binary_crossentropy', 'binary_crossentropy'], optimizer='adam')

    def train(self, epochs, batch_size, train_ds):
        for epoch in range(epochs):
            for x, y in train_ds:
                batch_size = x.shape[0]
                # Train Discriminator
                # Train on real images
                real_image = x
                real_latent = self.encoder.predict(real_image, verbose=0)
                d_loss_real_image = self.discriminator_image.train_on_batch(
                    real_image, np.ones((batch_size, 1)))
                d_loss_real_latent = self.discriminator_latent.train_on_batch(
                    real_latent, np.ones((batch_size, 1)))
                # Train on fake images
                latent = np.random.normal(0, 1, (batch_size, self.latent_dim))
                fake_image = self.generator.predict(latent, verbose=0)
                fake_latent = self.encoder.predict(fake_image, verbose=0)
                d_loss_fake_image = self.discriminator_image.train_on_batch(
                    fake_image, np.zeros((batch_size, 1)))
                d_loss_fake_latent = self.discriminator_latent.train_on_batch(
                    fake_latent, np.ones((batch_size, 1)))
                # Train Generator
                latent = np.random.normal(0, 1, (batch_size, self.latent_dim))
                g_loss = self.combined.train_on_batch(
                    latent, [np.ones((batch_size, 1)), np.ones((batch_size, 1))])
                print(
                    f'Epoch: {epoch+1}/{epochs}, d_loss_real_image: {d_loss_real_image[0]:.2f}, d_loss_real_latent: {d_loss_real_latent[0]:.2f}, d_loss_fake_image: {d_loss_fake_image[0]:.2f}, d_loss_fake_latent: {d_loss_fake_latent[0]:.2f}, g_loss: {g_loss[0]:.2f}')
            if epoch % 1 == 0:
                self.save_images(epoch)
                self.save_model()

    def save_images(self, epoch):
        r, c = 5, 5
        latent = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(latent)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig(f"v1/images/{epoch}.png")
        plt.close()

    def save_model(self):
        self.encoder.save('v1/encoder.h5')
        self.generator.save('v1/generator.h5')
        self.discriminator_image.save('v1/discriminator_image.h5')
        self.discriminator_latent.save('v1/discriminator_latent.h5')


aegan = AEGAN(encoder, generator, discriminator_image,
              discriminator_latent, latent_dim, input_shape)


In [25]:
aegan.train(epochs=10, batch_size=32, train_ds=train_ds)


Epoch: 1/10, d_loss_real_image: 1.56, d_loss_real_latent: 0.28, d_loss_fake_image: 0.68, d_loss_fake_latent: 0.59, g_loss: 1.52
Epoch: 1/10, d_loss_real_image: 0.94, d_loss_real_latent: 0.08, d_loss_fake_image: 0.82, d_loss_fake_latent: 0.48, g_loss: 1.05
Epoch: 1/10, d_loss_real_image: 0.00, d_loss_real_latent: 0.01, d_loss_fake_image: 1.05, d_loss_fake_latent: 0.35, g_loss: 0.93
Epoch: 1/10, d_loss_real_image: 0.00, d_loss_real_latent: 0.00, d_loss_fake_image: 1.14, d_loss_fake_latent: 0.23, g_loss: 0.89
Epoch: 1/10, d_loss_real_image: 0.00, d_loss_real_latent: 0.00, d_loss_fake_image: 1.02, d_loss_fake_latent: 0.12, g_loss: 0.88
Epoch: 1/10, d_loss_real_image: 0.00, d_loss_real_latent: 0.00, d_loss_fake_image: 0.78, d_loss_fake_latent: 0.05, g_loss: 0.83
Epoch: 1/10, d_loss_real_image: 0.00, d_loss_real_latent: 0.00, d_loss_fake_image: 0.78, d_loss_fake_latent: 0.02, g_loss: 0.78
Epoch: 1/10, d_loss_real_image: 0.00, d_loss_real_latent: 0.00, d_loss_fake_image: 0.79, d_loss_fake_lat

KeyboardInterrupt: 

In [42]:
# define the discriminator model
def define_discriminator(input_shape, num_classes):
    img_input = tf.keras.layers.Input(shape=input_shape)
    label_input = tf.keras.layers.Input(shape=(num_classes,))

    # image branch
    layer = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(img_input)
    layer = tf.keras.layers.LeakyReLU(alpha=0.2)(layer)
    layer = tf.keras.layers.Conv2D(
        128, (3, 3), strides=(2, 2), padding='same')(layer)
    layer = tf.keras.layers.LeakyReLU(alpha=0.2)(layer)
    layer = tf.keras.layers.Conv2D(
        128, (3, 3), strides=(2, 2), padding='same')(layer)
    layer = tf.keras.layers.LeakyReLU(alpha=0.2)(layer)
    layer = tf.keras.layers.Conv2D(
        256, (3, 3), strides=(2, 2), padding='same')(layer)
    layer = tf.keras.layers.LeakyReLU(alpha=0.2)(layer)
    layer = tf.keras.layers.Conv2D(
        256, (3, 3), strides=(2, 2), padding='same')(layer)
    layer = tf.keras.layers.LeakyReLU(alpha=0.2)(layer)
    layer = tf.keras.layers.Flatten()(layer)

    # label branch
    label_layer = tf.keras.layers.Dense(64)(label_input)
    label_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(label_layer)
    label_layer = tf.keras.layers.Dense(128)(label_layer)
    label_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(label_layer)
    label_layer = tf.keras.layers.Dense(256)(label_layer)
    # label_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(label_layer)
    # label_layer = tf.keras.layers.Dense(256)(label_layer)
    label_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(label_layer)
    label_layer = tf.keras.layers.Flatten()(label_layer)

    # concatenate image and label branches
    layer = tf.keras.layers.concatenate([layer, label_layer])
    layer = tf.keras.layers.Dropout(0.4)(layer)
    layer = tf.keras.layers.Dense(1, activation='sigmoid')(layer)

    # define the model
    model = tf.keras.models.Model(
        inputs=[img_input, label_input], outputs=layer)

    # compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

    return model


def define_aegenerator(latent_dim, num_classes, input_shape):
    # Encoder network
    encoder_input = tf.keras.layers.Input(shape=input_shape)
    encoder = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(encoder_input)
    encoder = tf.keras.layers.LeakyReLU(alpha=0.2)(encoder)
    encoder = tf.keras.layers.Conv2D(
        128, (3, 3), strides=(2, 2), padding='same')(encoder)
    encoder = tf.keras.layers.LeakyReLU(alpha=0.2)(encoder)
    encoder = tf.keras.layers.Conv2D(
        128, (3, 3), strides=(2, 2), padding='same')(encoder)
    encoder = tf.keras.layers.LeakyReLU(alpha=0.2)(encoder)
    encoder = tf.keras.layers.Conv2D(
        256, (3, 3), strides=(2, 2), padding='same')(encoder)
    encoder = tf.keras.layers.LeakyReLU(alpha=0.2)(encoder)
    encoder = tf.keras.layers.Conv2D(
        256, (3, 3), strides=(2, 2), padding='same')(encoder)
    encoder = tf.keras.layers.LeakyReLU(alpha=0.2)(encoder)
    encoder = tf.keras.layers.Flatten()(encoder)
    encoder = tf.keras.layers.Dense(latent_dim)(encoder)
    # Decoder network
    generator_input = tf.keras.layers.Input(shape=(latent_dim,))
    label_input = tf.keras.layers.Input(shape=(num_classes,))
    # Concatenate the latent representation and class label
    merge = tf.keras.layers.concatenate([generator_input, label_input])
    # Project to a larger dimension
    n = 128
    size = int(input_shape[0]/16)
    n_nodes = n * size * size
    dense = tf.keras.layers.Dense(n_nodes)(merge)
    # Reshape the dense layer output
    reshaped = tf.keras.layers.Reshape((size, size, n))(dense)
    # Upsample the feature maps
    upsampled = tf.keras.layers.Conv2DTranspose(
        256, (4, 4), strides=(2, 2), padding='same')(reshaped)
    upsampled = tf.keras.layers.LeakyReLU(alpha=0.2)(upsampled)
    upsampled = tf.keras.layers.Conv2DTranspose(
        256, (4, 4), strides=(2, 2), padding='same')(upsampled)
    upsampled = tf.keras.layers.LeakyReLU(alpha=0.2)(upsampled)
    upsampled = tf.keras.layers.Conv2DTranspose(
        128, (4, 4), strides=(2, 2), padding='same')(upsampled)
    upsampled = tf.keras.layers.LeakyReLU(alpha=0.2)(upsampled)
    upsampled = tf.keras.layers.Conv2DTranspose(
        128, (4, 4), strides=(2, 2), padding='same')(upsampled)
    upsampled = tf.keras.layers.LeakyReLU(alpha=0.2)(upsampled)
    # Output layer
    generator_output = tf.keras.layers.Conv2D(
        3, (3, 3), activation='tanh', padding='same')(upsampled)
    # Define the model
    generator = tf.keras.models.Model(
        [generator_input, label_input], generator_output)
    return generator


In [43]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    gen_noise, gen_label = g_model.input
    gen_output = g_model.output
    gan_output = d_model([gen_output, gen_label])
    model = tf.keras.Model([gen_noise, gen_label], gan_output)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [44]:
# build and compile the discriminator
discriminator = define_discriminator(input_shape, num_classes)
discriminator.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv2d_18 (Conv2D)             (None, 96, 96, 64)   1792        ['input_11[0][0]']               
                                                                                                  
 leaky_re_lu_29 (LeakyReLU)     (None, 96, 96, 64)   0           ['conv2d_18[0][0]']              
                                                                                                  
 conv2d_19 (Conv2D)             (None, 48, 48, 128)  73856       ['leaky_re_lu_29[0][0]']         
                                                                                            

In [45]:
# build the generator
generator = define_aegenerator(latent_dim, num_classes, input_shape)
generator.summary()


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 18)]         0           []                               
                                                                                                  
 concatenate_3 (Concatenate)    (None, 118)          0           ['input_14[0][0]',               
                                                                  'input_15[0][0]']               
                                                                                                  
 dense_18 (Dense)               (None, 4608)         548352      ['concatenate_3[0][0]']    

In [46]:
# build the gan
gan_model = define_gan(generator, discriminator)
gan_model.summary()


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 18)]         0           []                               
                                                                                                  
 concatenate_3 (Concatenate)    (None, 118)          0           ['input_14[0][0]',               
                                                                  'input_15[0][0]']               
                                                                                                  
 dense_18 (Dense)               (None, 4608)         548352      ['concatenate_3[0][0]']    

In [47]:
def generate_fake_samples(g_model, latent_dim, n_samples, n_classes):
    # generate points in latent space
    noise_input = generate_latent_points(latent_dim, n_samples)
    # generate random class labels
    y_types = generate_class_labels(n_samples, n_classes)
    # predict outputs
    X = g_model.predict([noise_input, y_types], verbose=0)
    # create 'fake' class labels (0)
    y_class = tf.zeros((n_samples, 1))
    return X, y_types, y_class


def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = tf.random.normal(shape=(n_samples, latent_dim))
    return x_input


def generate_class_labels(n_samples, n_classes):
    class_labels = np.zeros((n_samples, n_classes))
    for i in range(n_samples):
        class_labels[i, np.random.randint(0, n_classes)] = 1
        if np.random.random() > 0.5:
            class_labels[i, np.random.randint(0, n_classes)] = 1
    class_labels = tf.convert_to_tensor(class_labels, dtype=tf.float32)
    # print(class_labels)
    return class_labels


def generate_real_samples(dataset, n_samples):
    X, y_types, y_class = [], [], []
    while len(X) < n_samples:
        dataset.shuffle(100)
        for x, label in dataset.take(5):
            for i in range(len(x)):
                X.append(x[i])
                y_types.append(label[i][1:])
                y_class.append(label[i][0])
                if len(X) == n_samples:
                    break
            if len(X) == n_samples:
                break
    X = tf.stack(X)
    y_types = tf.stack(y_types)
    y_class = tf.reshape(tf.stack(y_class), (n_samples, 1))
    return X, y_types, y_class


In [48]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real_types, y_real_class = generate_real_samples(
        dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(
        [X_real, y_real_types], y_real_class, verbose=0)
    # prepare fake examples
    x_fake, y_fake_types, y_fake_class = generate_fake_samples(
        g_model, latent_dim, n_samples, 18)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(
        [x_fake, y_fake_types], y_fake_class, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' %
          (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'v2/aegenerator_model_64_e%03d.h5' % (epoch+1)
    g_model.save(filename)


def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # scale from [-1,1] to [0,1]
        examples[i, :, :, :] = (examples[i, :, :, :] + 1) / 2.0
        # plot raw pixel data
        plt.imshow(examples[i, :, :, :])
    # save plot to file
    filename = 'v2/aegenerated_plot_64_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()


In [49]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, resume_training=0):
    bat_per_epo = int(dataset.cardinality())
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j, batch in enumerate(dataset):
            # get randomly selected 'real' samples
            X_real, y_real_types, y_real_class = batch[0], batch[1][:,
                                                                    1:], batch[1][:, 0]
            half_batch = int(X_real.shape[0])
            # generate 'fake' examples
            X_fake, y_fake_types, y_fake_class = generate_fake_samples(
                g_model, latent_dim, half_batch, 18)
            # create training set for the discriminator
            y_real_class = tf.reshape(y_real_class, (half_batch, 1))
            X, y_types, y_class = tf.concat([X_real, X_fake], axis=0), tf.concat(
                [y_real_types, y_fake_types], axis=0), tf.concat([y_real_class, y_fake_class], axis=0)
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch([X, y_types], y_class)
            # prepare points in latent space as input for the generator
            X_gan, y_gan_types, y_gan_class = generate_latent_points(
                latent_dim, n_batch), generate_class_labels(n_batch, 18), tf.ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(
                [X_gan, y_gan_types], y_gan_class)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' %
                  (i+resume_training+1, j+1, bat_per_epo, d_loss, g_loss))
        # evaluate the model performance, sometimes
        if (i+resume_training) % 10 == 0:
            summarize_performance(i+resume_training, g_model,
                                  d_model, dataset, latent_dim)
            pass
    # save the generator model
    g_model.save('v2/aegenerator_model_64_final.h5')


In [51]:
train(generator, discriminator, gan_model, train_ds,
      latent_dim, n_epochs=11, resume_training=0)


>1, 1/433, d=0.697, g=0.691
>1, 2/433, d=0.666, g=0.692
>1, 3/433, d=0.635, g=0.691
>1, 4/433, d=0.603, g=0.687
>1, 5/433, d=0.564, g=0.682
>1, 6/433, d=0.524, g=0.662
>1, 7/433, d=0.496, g=0.619
>1, 8/433, d=0.512, g=0.544
>1, 9/433, d=0.572, g=0.465
>1, 10/433, d=0.689, g=0.427
>1, 11/433, d=0.737, g=0.448
>1, 12/433, d=0.720, g=0.520
>1, 13/433, d=0.683, g=0.598
>1, 14/433, d=0.652, g=0.669
>1, 15/433, d=0.639, g=0.751
>1, 16/433, d=0.626, g=0.822
>1, 17/433, d=0.596, g=0.898
>1, 18/433, d=0.595, g=0.965
>1, 19/433, d=0.567, g=1.014
>1, 20/433, d=0.555, g=1.010
>1, 21/433, d=0.515, g=1.040
>1, 22/433, d=0.465, g=1.204
>1, 23/433, d=0.427, g=1.368
>1, 24/433, d=0.396, g=1.340
>1, 25/433, d=0.402, g=1.265
>1, 26/433, d=0.319, g=1.985
>1, 27/433, d=0.264, g=1.690
>1, 28/433, d=0.268, g=1.348
>1, 29/433, d=0.230, g=2.858
>1, 30/433, d=0.143, g=2.659
>1, 31/433, d=0.252, g=0.771
>1, 32/433, d=0.404, g=3.381
>1, 33/433, d=0.169, g=3.982
>1, 34/433, d=0.209, g=3.147
>1, 35/433, d=0.204, g=